In [ ]:
import numpy as np
import tensorflow as tf
from model import Model
import os
from pykalman import KalmanFilter

In [ ]:
with open('data.txt') as f:
    first = f.readline()
    first = first.strip('\n')
    temp = first.split(' ')
    T = int(temp[0])
    o_dim = int(temp[1])
    s_dim = int(temp[2])
    o_matrix = np.zeros((T, o_dim), np.float32)
    for i in range(T):
        temp = f.readline().strip('\n').split(' ')
        for j in range(s_dim):
            o_matrix[i,j] = float(temp[j])
    s_matrix = np.zeros((T, s_dim), np.float32)
    for i in range(T):
        temp = f.readline().strip('\n').split(' ')
        for j in range(o_dim):
            s_matrix[i,j] = float(temp[j])

In [ ]:
T_train = 1000
train_data = np.zeros((T_train, 3*s_dim), np.float32)
for i in range(T_train):
    train_data[i, :] = np.concatenate((s_matrix[i, :], s_matrix[i+1, :], o_matrix[i+1, :]), axis=0)
    
T_test = 1000
test_data = np.zeros((T_test, 3*s_dim), np.float32)
for i in range(T_test):
    test_data[i, :] = np.concatenate((s_matrix[T_train + i, :], s_matrix[T_train + 1 + i, :], o_matrix[T_train + 1 + i, :]),0)

In [ ]:
train_T = train_data.shape[0]
s_t_minus_1_train = train_data[:,:s_dim]
s_t_train = train_data[:,s_dim:2*s_dim]

sumStSt_1 = np.sum(\
                   np.matmul(\
                             np.reshape(s_t_train, [-1,s_dim,1]),\
                             np.reshape(s_t_minus_1_train, [-1,1,s_dim])\
                            ),axis=0\
                  )
sumSt_1 = np.transpose(np.sum(s_t_minus_1_train, axis=0, keepdims=True))
sumSt = np.transpose(np.sum(s_t_train,axis=0,keepdims=True))
sumSt_1St_1 =np.sum(\
                    np.matmul(\
                              np.reshape(s_t_minus_1_train, [-1,s_dim,1]),\
                              np.reshape(s_t_minus_1_train, [-1,1,s_dim])\
                             ),axis=0\
                   )

A_2 = np.matmul(\
                sumStSt_1 - (np.matmul(sumSt, np.transpose(sumSt_1)) / train_T),\
                np.linalg.inv(\
                              sumSt_1St_1 - (np.matmul(sumSt_1, np.transpose(sumSt_1)) / train_T)
                             )\
               )
b_2 = (sumSt - np.matmul(A_2, sumSt_1)) / train_T

tmp = s_t_train - np.matmul(s_t_minus_1_train, np.transpose(A_2)) - np.repeat(b_2.reshape([1,-1]),train_T,axis=0)
Sig_2 = np.mean(\
                np.matmul(\
                          np.reshape(tmp,[-1,s_dim,1]),\
                          np.reshape(tmp,[-1,1,s_dim])\
                         ),axis=0\
               )

o_t_train = train_data[:,2*s_dim:3*s_dim]
sumOtSt = np.sum(\
                 np.matmul(\
                           np.reshape(o_t_train,[-1,o_dim,1]),\
                           np.reshape(s_t_train,[-1,1,s_dim])\
                          ),axis=0\
                )
sumStSt = np.sum(\
                 np.matmul(\
                           np.reshape(s_t_train,[-1,s_dim,1]),\
                           np.reshape(s_t_train,[-1,1,s_dim])\
                          ),axis=0\
                )
sumOt = np.transpose(np.sum(o_t_train,axis=0,keepdims=True))

A_3 = np.matmul(\
                sumOtSt - (np.matmul(sumOt, np.transpose(sumSt)) / train_T),\
                np.linalg.inv(\
                              sumStSt - (np.matmul(sumSt, np.transpose(sumSt)) / train_T)
                             )\
               )
b_3 = (sumOt - np.matmul(A_3, sumSt)) / train_T

tmp = o_t_train - np.matmul(s_t_train, np.transpose(A_3)) - np.repeat(b_3.reshape([1,-1]),train_T,axis=0)
Sig_3 = np.mean(\
                np.matmul(\
                          np.reshape(tmp,[-1,o_dim,1]),\
                          np.reshape(tmp,[-1,1,o_dim])\
                         ),axis=0\
               )

In [ ]:
kf_orig = KalmanFilter(\
                       initial_state_mean =\
                           np.matmul(A_2, np.transpose(test_data[0,:s_dim])) + b_2.reshape(-1),\
                       initial_state_covariance = Sig_2,\
                       transition_matrices = A_2, \
                       transition_covariance = Sig_2, \
                       transition_offsets = np.transpose(b_2.reshape(-1)),\
                       observation_matrices = A_3,\
                       observation_covariance = Sig_3,\
                       observation_offsets = np.transpose(b_3.reshape(-1))\
                      )
measurements = test_data[:,2*s_dim:]
(est_s_t, est_s_t_covariances) = kf_orig.filter(measurements)
print(np.mean(np.linalg.norm(est_s_t - test_data[:, s_dim : 2*s_dim] , axis = 1)), end = '')
print(' / ', end = '')
print(np.mean(np.linalg.norm(test_data[:, s_dim : 2*s_dim], axis = 1)))
print('mean consecutive diff: ', end='')
print(np.mean(np.linalg.norm(test_data[1:, s_dim : 2*s_dim] - test_data[:-1, s_dim : 2*s_dim], axis = 1)))

In [ ]:
class Dataset(object):
    def __init__(self, train, test):
        self.train = train
        self.test = test
    def random_batch(self, batch_size):
        index = np.random.choice(np.arange(len(self.train)),batch_size, False)
        return self.train[index,:]
dataset = Dataset(train_data, test_data)

In [ ]:
log_dir = './log/'
os.popen('rm '+log_dir+'*')
h_dim = 10
minibatch_size = 128
model = Model(s_dim, h_dim, minibatch_size, 1e-4, log_dir)
iteration = 300
for epoch in range(100):
    reconstruction_loss_train, likelihood_train, classify_loss_train = 0., 0., 0.
    global_step = tf.contrib.framework.get_or_create_global_step()
    # in each epoch 500 iterations
    for i in range(iteration):
        reconstruction_loss_value, likelihood_value, classify_loss_value, summary = \
                model.update_params(dataset.random_batch(minibatch_size))
            
        reconstruction_loss_train += reconstruction_loss_value
        likelihood_train += likelihood_value
        classify_loss_train += classify_loss_value
        model.train_writer.add_summary(summary, global_step.eval(model.sess))
    
    reconstruction_loss_train = reconstruction_loss_train / (iteration)
    likelihood_train = -likelihood_train / (iteration)
    classify_loss_train = classify_loss_train / (iteration)
    

    print("step: {},\trecons loss: {:.4f},\tlikelihood: {:.4f},\tclass loss: {:.4f}".format(global_step.eval(model.sess),
            reconstruction_loss_train, likelihood_train, classify_loss_train))


In [ ]:
last_s_p = np.asarray(model.sess.run([model.s_t_p], {model.input_tensor: train_data[train_data.shape[0]-2 : ,:]}))[0][1,:]

a_2, b_2, sig_2, a_3, b_3, sig_3 = model.sess.run([model.a_2, model.b_2, model.sigma_2,\
                                                    model.a_3, model.b_3, model.sigma_3],\
                                                   {model.input_tensor: train_data}\
                                                  )
kf = KalmanFilter(initial_state_mean = np.transpose(np.matmul(last_s_p,a_2) + b_2), \
                  initial_state_covariance = sig_2, \
                  transition_matrices = np.transpose(a_2), \
                  transition_offsets = b_2,
                  transition_covariance = sig_2, \
                  observation_matrices = np.transpose(a_3),\
                  observation_offsets = b_3,
                  observation_covariance = sig_3)
est_o_t_p = model.sess.run(model.o_t_p, {model.input_tensor: test_data})
measurements = np.asarray(est_o_t_p)
(est_s_t_p, est_s_t_p_covariances) = kf.filter(measurements)
est_s_t = model.decode_s_t_p(est_s_t_p)
print(np.mean(np.linalg.norm(est_s_t - test_data[:, s_dim : 2*s_dim] , axis = 1)), end = '')
print(' / ', end = '')
print(np.mean(np.linalg.norm(test_data[:, s_dim : 2*s_dim], axis = 1)))
print('mean consecutive diff: ', end='')
print(np.mean(np.linalg.norm(test_data[1:, s_dim : 2*s_dim] - test_data[:-1, s_dim : 2*s_dim], axis = 1)))